In [1]:
# 探索初始蒙特卡洛算法
import random
import time
from yuanyangEnv import YuanYangEnv
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.9.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class MC_RL:
    def __init__(self,yuanyang):
        # 行为值函数的初始化
        self.qvalue = np.ones((len(yuanyang.states),len(yuanyang.actions)))*0.1
        
        # 次数初始化，求经验平均时，q(s,a)=G(s,a)/n(s,a)
        self.n = 0.001*np.ones((len(yuanyang.states),len(yuanyang.actions)))
        self.actions = yuanyang.actions
        self.gamma = yuanyang.gamma
        self.yuanyang = yuanyang

    def greedy_policy(self,qfun,state):
        # 贪婪策略
        amax = qfun[state,:].argmax()
        return self.actions[amax]

    def epsilon_greedy_policy(self,qfun,state,epsilon):
        # ε-greedy策略
        amax = qfun[state,:].argmax()
        if np.random.uniform() < 1 - epsilon:
            return self.greedy_policy(qfun,state)
        else:
            return self.actions[int(np.random.rand()*len(self.actions))]

    def find_anum(self,a):
        # 找到动作对应的序号
        for i in range(len(self.actions)):
            if a == self.actions[i]:
                return i

    def mc_learning_ei(self, num_iter):
        # 学习num_iter次
        self.qvalue = np.zeros((len(yuanyang.states), len(yuanyang.actions)))
        self.n = 0.001 * np.ones((len(yuanyang.states), len(yuanyang.actions)))
        for iter in range(num_iter):
            # 采集状态样本
            s_sample = []
            
            # 采集动作样本
            a_sample = []
            
            # 采集回报样本
            r_sample = []
            
            # 随机初始化状态
            s = self.yuanyang.reset()
            a = self.actions[int(random.random() * len(self.actions))]
            done = False
            step_num = 0
            if self.mc_test() == 1:
                print("探索初始化第一次完成任务需要的次数：", iter)
                break
                
            # 采集数据s0-a1-s1-a2-s2...terminate state
            # for i in range(5):
            while False == done and step_num < 30:
                # 与环境交互
                s_next, r, done = self.yuanyang.transform(s, a)
                a_num = self.find_anum(a)
                
                # 往回走给予惩罚
                if s_next in s_sample:
                    r = -2
                    
                # 存储数据，采样数据
                s_sample.append(s)
                r_sample.append(r)
                a_sample.append(a_num)
                step_num += 1
                
                # 转移到下一个状态，继续试验，s0-s1-s2
                s = s_next
                a = self.greedy_policy(self.qvalue, s)
                
            # 从样本中计算累计回报,g(s_0) = r_0+gamma*r_1+gamma^2*r_2+gamma^3*r3+v(sT)
            a = self.greedy_policy(self.qvalue, s)
            g = self.qvalue[s, self.find_anum(a)]
            
            # 计算该序列的第一状态的累计回报
            for i in range(len(s_sample) - 1, -1, -1):
                g *= self.gamma
                g += r_sample[i]
                
            # print("episode number, trajectory", iter1, s_sample)
            # print("first state", s_sample[0], g)
            # g=G(s1,a),开始算其他状态处的累计回报
            for i in range(len(s_sample)):
                # 计算状态-行为对（s,a)的次数，s,a1...s,a2
                self.n[s_sample[i], a_sample[i]] += 1.0
                
                # 利用增量式方法更新值函数
                self.qvalue[s_sample[i], a_sample[i]] = (self.qvalue[s_sample[i], a_sample[i]] * (
                        self.n[s_sample[i], a_sample[i]] - 1) + g) / self.n[s_sample[i], a_sample[i]]
                
                g -= r_sample[i]
                g /= self.gamma
                # print("s_sample,a",g)
            # print("number",self.n)
        return self.qvalue

    def mc_test(self):
        # 初始状态为0，智能体通过贪婪策略与环境进行交互，如果结束时，智能体找到了目标，则设置标志位为1.否则为0
        s = 0
        s_sample = []
        done = False
        flag = 0
        step_num = 0
        while False == done and step_num < 30:
            a = self.greedy_policy(self.qvalue, s)
            
            # 与环境交互
            s_next, r, done = self.yuanyang.transform(s, a)
            s_sample.append(s)
            s = s_next
            step_num += 1
        if s == 9:
            flag = 1    # 到达目标
        return flag

In [ ]:
yuanyang = YuanYangEnv()
brain = MC_RL(yuanyang)

# 探索初始化方法
qvalue1 = brain.mc_learning_ei(num_iter=1000000)

# 将行为值函数渲染出来
yuanyang.action_value = qvalue1
print(qvalue1)

# 测试学到的策略
flag = 1
s = 0
# print(policy_value.pi)
step_num = 0
path = []

# 将最优路径打印出来
while flag:
    # 渲染路径点
    path.append(s)
    yuanyang.path = path
    a = brain.greedy_policy(qvalue1, s)
    # a = agent.bolzman_policy(qvalue,s,0.1)
    print('%d->%s\t' % (s, a), qvalue1[s, 0], qvalue1[s, 1], qvalue1[s, 2], qvalue1[s, 3])
    yuanyang.bird_male_position = yuanyang.state_to_position(s)
    yuanyang.render()
    time.sleep(0.25)
    step_num += 1
    s_, r, t = yuanyang.transform(s, a)
    if t == True or step_num > 30:
        flag = 0
    s = s_
    
# 渲染最后的路径点
yuanyang.bird_male_position = yuanyang.state_to_position(s)
path.append(s)
yuanyang.render()
while True:
    yuanyang.render()

探索初始化第一次完成任务需要的次数： 1663
[[-28.51974222 -27.1942155  -27.47661624 -28.25840333]
 [-27.09702259 -27.38328463 -28.45194141 -28.05795824]
 [-26.99828817 -28.38683713 -30.03129832 -26.06504482]
 [  0.           0.           0.           0.        ]
 [-26.52909933 -26.50962555 -26.00646373 -25.74092779]
 [-24.22991433 -25.50401748 -26.91235621 -24.02953407]
 [  0.           0.           0.           0.        ]
 [  9.37131601   6.4824196   -1.14023037  -1.19815375]
 [  9.99997955   6.73056574   7.02359528  -0.50003813]
 [  0.           0.           0.           0.        ]
 [-28.04100336 -20.01099163 -28.00803767 -27.52861208]
 [-27.24268874 -27.85496351 -27.73694774 -27.29772112]
 [-26.70488943 -27.88780762 -27.29050784 -28.35974003]
 [  0.           0.           0.           0.        ]
 [-27.79828969 -29.41393067 -27.43543385 -27.27075827]
 [-26.47208645 -26.60050892 -28.67370679 -26.22673966]
 [  0.           0.           0.           0.        ]
 [  2.81239587   3.20540633  -5.85853735 